In [1]:
from tensorflow.python.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Embedding, LSTM, Concatenate
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import pandas as pd
import numpy as np

In [2]:
# encoding은 보통 utf-8, cp949 로 하면되지만 이번 파일은 latin1
data = pd.read_csv('1.3만_워크맨_m8JrVquHNsY.csv', header = None, names = ['review','date','like'])
data2 = pd.read_csv('1.6만_워크맨_UQYCWLrCUy4.csv', header = None, names = ['review','date','like'])
data3 = pd.read_csv('2.4만_워크맨_BWROdI-AgAU.csv', header = None, names = ['review','date','like'])

data_all = pd.concat([data,data2,data3])

data_all

,review,date,like
0,🐜🐜워크맨 인력소🐜🐜\r\n워크맨의 리뷰를 원하는 job것이 있다면↓댓글 추천↓\r...,1년 전,7101
1,쿠팡상하차,1년 전,1055
2,야구장 치어리더,1년 전,823
3,AV 배우,1년 전,627
4,임상실험 알바 해주세요!!!!,1년 전,223
...,...,...,...
3679,아나운서였오 미친ㅋㅋㅋ,6개월 전,0
3680,똘끼 쩌네요 ㅋ\n편집점이 이렇게 나오게 하는건지,6개월 전,1
3681,아침마다 날씨로 뵙는분인데 이렇게 웃기신 분일줄이얔ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,6개월 전,0
3682,저슬 쭈물러욬ㅋㅋㅋㅋㅋ,6개월 전,0


In [3]:
data_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10409 entries, 0 to 3683
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  10409 non-null  object
 1   date    10409 non-null  object
 2   like    10409 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 325.3+ KB


In [4]:
data_all['date'].unique()

array(['1년 전', '1년 전(수정됨)', '10개월 전', '9개월 전', '8개월 전', '7개월 전', '6개월 전',
       '4개월 전', '4개월 전(수정됨)', '3개월 전', '2개월 전', '5개월 전(수정됨)', '2주 전',
       '1주 전', '1개월 전', '4주 전', '9개월 전(수정됨)', '3주 전', '3주 전(수정됨)', '4일 전',
       '34분 전', '30분 전', '5개월 전', '8개월 전(수정됨)', '3시간 전', '11개월 전',
       '7개월 전(수정됨)', '14시간 전', '11개월 전(수정됨)', '1개월 전(수정됨)', '3개월 전(수정됨)',
       '9시간 전', '2일 전', '10개월 전(수정됨)', '6개월 전(수정됨)', '3일 전', '2개월 전(수정됨)',
       '1일 전', '6일 전', '5일 전', '16시간 전', '1주 전(수정됨)', '7시간 전',
       '2주 전(수정됨)', '10시간 전', '8시간 전', '13시간 전', '2시간 전', '4일 전(수정됨)'],
      dtype=object)

In [5]:
data_all['like'].mean()

100.15621097127486

In [6]:
Mean = data_all['like'].mean()
print(Mean)

data_all.loc[data_all["like"] < Mean, "like"] = 0
data_all.loc[data_all["like"] >= Mean, "like"] = 1

100.15621097127486


In [7]:
data_all['like'].value_counts()

0    9515
1     894
Name: like, dtype: int64

In [8]:
data_all['review']

0       🐜🐜워크맨 인력소🐜🐜\r\n워크맨의 리뷰를 원하는 job것이 있다면↓댓글 추천↓\r...
1                                                   쿠팡상하차
2                                                야구장 치어리더
3                                                   AV 배우
4                                        임상실험 알바 해주세요!!!!
                              ...                        
3679                                         아나운서였오 미친ㅋㅋㅋ
3680                          똘끼 쩌네요 ㅋ\n편집점이 이렇게 나오게 하는건지
3681            아침마다 날씨로 뵙는분인데 이렇게 웃기신 분일줄이얔ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ
3682                                         저슬 쭈물러욬ㅋㅋㅋㅋㅋ
3683                                        이누나 ㄹㅇ 취저네ㅋㅋㅋ
Name: review, Length: 10409, dtype: object

In [9]:
data_all['review'] = data_all['review'].str.replace("[^\w]", " ")
data_all['review'] = data_all['review'].str.replace("[0-9]", "")
data_all['review'] = data_all['review'].str.replace("[a-z]", "")
data_all['review'] = data_all['review'].str.replace("[A-Z]", "")
data_all['review'] = data_all['review'].str.strip()
data_all['review'] = data_all['review'].replace('', np.nan)
data_all = data_all.dropna(how='any')

In [10]:
data_all['date'] = data_all['date'].str.replace("[^\w]", " ")
data_all['date'] = data_all['date'].str.replace("전", "")
data_all['date'] = data_all['date'].str.replace("개", "")
data_all['date'] = data_all['date'].str.strip()

<ipython-input-10-5c6af552bb78>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_all['date'] = data_all['date'].str.replace("[^\w]", " ")
<ipython-input-10-5c6af552bb78>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_all['date'] = data_all['date'].str.replace("전", "")
<ipython-input-10-5c6af552bb78>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

In [11]:
#data_all['review']=data_all[['review','date']].apply(lambda x: ' '.join(x), axis=1)

In [12]:
data_all['like'].value_counts()

0    8968
1     866
Name: like, dtype: int64

In [13]:
from sklearn.model_selection import train_test_split
review_train, review_test,date_train, date_test, y_train, y_test = train_test_split(data_all['review'],data_all['date'], data_all['like'], test_size=0.2, shuffle=False)

In [14]:
from konlpy.tag import Okt

stopwords = ['이', '가', '을', '를','은','는','에','의','도']

cnt = 0
X_train = []
for stc in review_train:
    words = Okt().morphs(stc)
    token = []
    for word in words:
        if word not in stopwords:
            token.append(word)
    X_train.append(token)
    
X_test = []
for stc in review_test:
    words = Okt().morphs(stc)
    token = []
    for word in words:
        if word not in stopwords:
            token.append(word)
    X_test.append(token)    



In [15]:
    
X_train_2 = []
for stc in date_train:
    X_train_2.append(stc.split())
    
X_test_2 = []
for stc in date_test:
    X_test_2.append(stc.split())
    

In [16]:
print(len(X_train))
print(len(X_test))
print(len(X_train_2))
print(len(X_test_2))

for i in range(len(X_train)):
    X_train[i]+=X_train_2[i]
for i in range(len(X_test)):
    X_test[i]+=X_test_2[i]

7867
1967
7867
1967


In [17]:
# X_train_2 = []
# for stc in date_train:
#     X_train_2.append(stc.split())
    
# X_test_2 = []
# for stc in date_test:
#     X_test_2.append(stc.split())

In [18]:
from tensorflow.keras.preprocessing.text import Tokenizer

# X_train 단어들을 토대로 정수 인덱스 설정
# 빈도수가 높은 것부터 4000개만 정수 인덱스로 변환하겠다!
tokenizer = Tokenizer(4000)
tokenizer.fit_on_texts(X_train)

# 위에서 설정된 정수 인덱스를 토대로 변환
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)


print('# int_encoding done')


# int_encoding done


In [19]:
# tokenizer_2 = Tokenizer()
# tokenizer_2.fit_on_texts(X_train_2)

# X_train_2 = tokenizer_2.texts_to_sequences(X_train_2)
# X_test_2 = tokenizer_2.texts_to_sequences(X_test_2)

In [20]:
print(len(tokenizer.word_index))

low_count = 0
for word, word_count in tokenizer.word_counts.items():
    if word_count == 1:
        low_count += 1
print(low_count)

13139
8319


In [21]:
max_length = 0
for data in X_train:
    if max_length < len(data):
        max_length = len(data)
print(max_length)

176


In [22]:
len(X_test)

1967

In [23]:
# len(X_test_2)

In [24]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_len = 40
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)
# X_train_2 = pad_sequences(X_train_2, maxlen=2)
# X_test_2 = pad_sequences(X_test_2, maxlen=2)

In [25]:
# X_train : np.concatenate((X_train_2,X_train), axis=1)
# X_test : np.concatenate((X_test_2,X_test), axis=1)

In [26]:
model = Sequential()
model.add(Embedding(4000, 32))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

In [27]:
# 혹시 5회 이상 검증데이터 loss가 증가하면, 과적합될 수 있으므로 학습을 조기종료!
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
# 훈련을 거듭하면서, 가장 검증데이터 정확도가 높았던 순간을 체크포인트로 저장
model_check = ModelCheckpoint('the_best.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [28]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['acc'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)

Epoch 1/10
246/246 [==============================] - 3s 10ms/step - loss: 0.2584 - acc: 0.9162 - val_loss: 0.2830 - val_acc: 0.8937
Epoch 2/10
246/246 [==============================] - 2s 10ms/step - loss: 0.2036 - acc: 0.9241 - val_loss: 0.2564 - val_acc: 0.8963
Epoch 3/10
246/246 [==============================] - 3s 11ms/step - loss: 0.1855 - acc: 0.9310 - val_loss: 0.2825 - val_acc: 0.8963
Epoch 4/10
246/246 [==============================] - 3s 12ms/step - loss: 0.1733 - acc: 0.9367 - val_loss: 0.2712 - val_acc: 0.9024
Epoch 5/10
246/246 [==============================] - 3s 14ms/step - loss: 0.1634 - acc: 0.9418 - val_loss: 0.2872 - val_acc: 0.9044
Epoch 6/10
246/246 [==============================] - 4s 15ms/step - loss: 0.1555 - acc: 0.9455 - val_loss: 0.2911 - val_acc: 0.9024
Epoch 7/10
246/246 [==============================] - 3s 13ms/step - loss: 0.1517 - acc: 0.9448 - val_loss: 0.2952 - val_acc: 0.9029
Epoch 8/10
246/246 [==============================] - 2s 9ms/step - l

In [71]:
# 정확도 측정
print(model.evaluate(X_test, y_test))

sentence = input()
date = input()

# 토큰화
sentence = sentence.replace("[^\w]", " ")
sentence = sentence.replace("[0-9]", "")
sentence = sentence.replace("[a-z]", "")
sentence = sentence.replace("[A-Z]", "")
sentence = sentence.strip()

date = date.replace("[^\w]", " ")
date = date.replace("전", "")
date = date.replace("개", "")
date = date.strip()

sentence+=date

words = Okt().morphs(sentence)

words_re = []
for word in words:
    if word not in stopwords:
        words_re.append(word)

encode_stc = tokenizer.texts_to_sequences([words_re])

pad_stc = pad_sequences(encode_stc, maxlen = 40)
print(pad_stc)
print(model.predict(pad_stc))

62/62 [==============================] - 0s 2ms/step - loss: 0.3276 - acc: 0.9014
[0.32758110761642456, 0.9013726711273193]


 진짜 이 편은 개인적으로 레전드다ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 선배님들도 그렇고 특히 미리 선배님이랑 점장님 센스 개 오지고 이때 장성규 입담 유독 내 스타일이기도 하고 개 웃기네ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ
 1년


[[   0    0    0    0    0    0    0    0    0    0    9   88  671   71
    38  251   22  453  359   21  636 1784  359   21  149  478   39  697
    52   49    6 1386   87 1510   64   39   91   40  826    1]]
[[0.10060126]]
